In [8]:
from google.colab import files
uploadedFile = files.upload()

Saving Face_Recognition.json to Face_Recognition.json


In [0]:
import pandas as pd
import numpy as np
import cv2
import urllib.request
import io
import os,sys
from PIL import Image
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
K.set_image_dim_ordering('tf') 
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import Flatten
import dlib
import random
import csv
import pickle

In [0]:

data = pd.read_json('Face_Recognition.json',lines = True)

face_data = open('Face_Recog.csv', 'w')
csvwriter = csv.writer(face_data)
csvwriter.writerow(['Content', 'Features','Coords'])

for num in range(len(data)):
  URL = data["content"][num]
  for value in data['annotation'][num]:
    if ( len(value['label']) < 4 or value['label'][0] == 'Not_Face' ):
      continue
    a = value['label'][0].split('_')[-1].strip()
    b = value['label'][1].split('_')[-1].strip()
    c = value['label'][2].split('_')[-1].strip()
    d = value['label'][3].split('_')[-1].strip()
    startX = int(round(value['points'][0]['x'] * value['imageWidth']))
    startY = int(round(value['points'][0]['y'] * value['imageHeight']))
    endX = int(round(value['points'][1]['x'] * value['imageWidth']))
    endY = int(round(value['points'][1]['y'] * value['imageHeight']))
    csvwriter.writerow((URL,str(a+" "+b+" "+c+" "+d),(startX,startY,endX,endY)))
    
face_data.close()

In [0]:

csvdata = pd.read_csv('Face_Recog.csv')

with open('Face_Recog.csv', 'r') as f:
    reader = csv.reader(f)

    tempNP = []
    for row in reader:
        if row[0] != 'Content':
          URL = row[0]
          coords = row[2]
          coords = coords[1:-1].split(",")
          x1 = int(coords[0])
          y1 = int(coords[1])
          x2 = int(coords[2])
          y2 = int(coords[3])
          with urllib.request.urlopen(URL) as url:
            f = io.BytesIO(url.read())
          pil_image = Image.open(f).convert('RGB')
          cv2_image = cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)
          roi = cv2_image[y1:y2,x1:x2]
          npImage = np.array(roi,'float32')
          npImage /= 255
          npImage = cv2.resize(npImage,(48,48))
          tempNP.append(npImage)

In [12]:
images = np.array(tempNP)
images.shape

(198, 48, 48, 3)

In [0]:
Y = []
features = csvdata.drop(['Content','Coords'],axis=1)
features = np.array(features)
for i in range(len(features)):
  Y.append(features[i][0].split())

In [0]:
one_hot = MultiLabelBinarizer()
labels = one_hot.fit_transform(Y)

In [0]:
xtrain, xtest, ytrain, ytest = train_test_split(images, labels, random_state=42, test_size=0.2)

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(48,48,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(1, 1), activation='relu'))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(16, activation='sigmoid'))

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 46, 46, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 23, 23, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 21, 21, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 64)          8256      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 6, 6, 128)         73856     
__________

In [32]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.fit(xtrain, ytrain, validation_data=(xtest, ytest), nb_epoch=20, batch_size=4)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 158 samples, validate on 40 samples
Epoch 1/20
158/158 [==============================] - 1s 8ms/step - loss: 0.5086 - acc: 0.7389 - val_loss: 0.4460 - val_acc: 0.7797
Epoch 2/20
158/158 [==============================] - 0s 2ms/step - loss: 0.4615 - acc: 0.7603 - val_loss: 0.4447 - val_acc: 0.7875
Epoch 3/20
158/158 [==============================] - 0s 2ms/step - loss: 0.4583 - acc: 0.7690 - val_loss: 0.4263 - val_acc: 0.7875
Epoch 4/20
158/158 [==============================] - 0s 2ms/step - loss: 0.4584 - acc: 0.7678 - val_loss: 0.4374 - val_acc: 0.7922
Epoch 5/20
158/158 [==============================] - 0s 2ms/step - loss: 0.4540 - acc: 0.7650 - val_loss: 0.4271 - val_acc: 0.7875
Epoch 6/20
158/158 [==============================] - 0s 2ms/step - loss: 0.4484 - acc: 0.7714 - val_loss: 0.4320 - val_acc: 0.7703
Epoch 7/20
158/158 [==============================] - 0s 2ms/step - loss: 0.4417 - acc: 0.7848 - val_loss: 0.4437 - val_acc: 0.7625
Epoch 8/20
158/158 [===========

In [0]:
pickle.dump(model, open('facedata.sav', 'wb'))

In [0]:
IMAGE_NAME = 'image.jpeg'
image = cv2.imread(IMAGE_NAME)
inputs = []

if image is None:
    print("ERROR: No input image given.")
    exit()

try:
  cnn_face_detector = dlib.cnn_face_detection_model_v1('mmod_human_face_detector.dat')
except:
  print('Face detector module not found.')

faces_cnn = cnn_face_detector(image, 1)
if (len(faces_cnn)) < 1:
  print('No faces found in the image')
else:
  for face in faces_cnn:
      x = face.rect.left()
      x = int(x - (0.1 * x))
      y = face.rect.top()
      y = int(y - (0.15 * y))
      x2 = face.rect.right()
      x2 = int(x2 + (0.1 * x2))
      y2 = face.rect.bottom()
      y2 = int(y2 + (0.1 * y2))
      roi = image[y:y2,x:x2]
      inputs.append(roi)

In [34]:
age = ['below20','30','40','50'] ; ageprob = ['','','','']
gender = ['Male','Female'] ; genderprob = ['','']
emotion = ['Angry','Happy','Sad','Neutral'] ; emotionprob = ['','','','']
ethincity = ['Black','White','Asian','Indian','Hispanic'] ; ethincityprob = ['','','','','']
 
for img in inputs:
    img = np.array(img,'float32')
    img /= 255
    img = cv2.resize(img,(48,48))
    classes = np.array(one_hot.classes_)
    probability = model.predict(img.reshape(1,48,48,3))[0]
    for i in range(16):
      if classes[i] in age:
        ageprob[age.index(classes[i])] = probability[i]
      if classes[i] in gender:
        genderprob[gender.index(classes[i])] = probability[i]
      if classes[i] in emotion:
        emotionprob[emotion.index(classes[i])] = probability[i]
      if classes[i] in ethincity:
        ethincityprob[ethincity.index(classes[i])] = probability[i]
    print(age[ageprob.index(max(ageprob))])
    print(gender[genderprob.index(max(genderprob))])
    print(emotion[emotionprob.index(max(emotionprob))])
    print(ethincity[ethincityprob.index(max(ethincityprob))])
    print('---------------------------------------------')

    

40
Male
Neutral
Black
30
Female
Neutral
White
